# MosesAI conversation

1. Ask a question
2. Get an answer
3. Now ask the next question with history

In [1]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
MODEL="gpt-4"

In [2]:
import pinecone
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain

/home/mark/anaconda3/envs/MosesAI/lib/python3.9/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [7]:
import openai
from langchain.embeddings.openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(openai_api_key=openai.api_key)

query_result = embeddings.embed_query("Hello world")
len(query_result)

1536

In [8]:
import pinecone 
from langchain.vectorstores import Pinecone
# initialize pinecone
pinecone.init(
    api_key=os.getenv('PINECONE_API_KEY'), 
    environment="us-central1-gcp"  # next to api key in console
)
index_name = "talmud-pages"
index = Pinecone.from_existing_index(index_name, embeddings)

In [9]:
def get_similiar_docs(query,k=5,score=False):
  if score:
    similar_docs = index.similarity_search_with_score(query,k=k)
  else:
    similar_docs = index.similarity_search(query,k=k)
  return similar_docs

query = "When do you say Shema?"
similar_docs = get_similiar_docs(query)
len(similar_docs)
similar_docs[0]

Document(page_content='However, the Shema is not said when one actually goes to sleep or wakes up, but rather in the general period when people lie down and get up. When is this? In the evening - when the Kohanim, who were impure, are returning from the mikveh to eat priestly portion (terumah), that is, at nightfall. That is when the time to say the Shema in the evening begins, but when does it end? Rabbi Eliezer said, "Until the end of the first watch, that is, the first part of the night." Rabbi Eliezer understands "when you lie down" as the time when people go to sleep. The Sages say until midnight, and Rabban Gamliel says until dawn. We can understand Rabban Gamliel, \'when people lie down" means when people are asleep. But the opinion of the Sages will require clarification.', metadata={'source': '../data/talmud-paragraphs/brachot2.html-paragraph-1.txt'})

In [10]:
from langchain.llms import OpenAI

llm = OpenAI(model_name=MODEL)

/home/mark/anaconda3/envs/MosesAI/lib/python3.9/site-packages/langchain/llms/openai.py:202: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/home/mark/anaconda3/envs/MosesAI/lib/python3.9/site-packages/langchain/llms/openai.py:790: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


https://python.langchain.com/en/latest/use_cases/question_answering.html

In [11]:
from langchain.chains.question_answering import load_qa_chain
chain = load_qa_chain(llm, chain_type="stuff")

def get_answer(query):
  similar_docs = get_similiar_docs(query)
  # print(similar_docs)
  answer =  chain.run(input_documents=similar_docs, question=query)
  return  answer

query = "When to say Shema?"  
get_answer(query)

'The Shema prayer should be said twice a day, once in the morning and once in the evening. The time to say the Shema in the morning is "when people rise," or when there is enough light to distinguish between blue and white wool, a wolf and a dog, or to recognize a friend from four steps away. According to Rabbi Eliezer the time to say the Shema in the morning ends at sunrise, but Rabbi Yehoshua states that it ends three hours into the day.\n\nThe time to say the Shema in the evening begins when the Kohanim, who were impure, are returning from the mikveh to eat their priestly portion (terumah), or at nightfall. The time it ends varies according to different opinions: Rabbi Eliezer states that it should be said until the end of the first watch (first part of the night), the Sages say it should be said until midnight, and Rabban Gamliel says it should be said until dawn.'